In [133]:
from libgen_api import LibgenSearch
import pandas as pd
import fitz
import traceback
from IPython.display import clear_output
# https://pypi.org/project/libgen-api/

In [137]:
def scrape_books():
    s = LibgenSearch()
    df = pd.DataFrame(columns = ['ID', 'Author', 'Title', 'Publisher', 'Year', 'Pages', 'Language', 'Size', 'Extension', 'Mirror_1', 'Mirror_2',  'Mirror_3', 'Mirror_4', 'Mirror_5', 'Edit'])
    try:
        x = True
        while True:
            search_input = input('How do you want to search? (author, title, or both)')
            
            
            if search_input == 'author':
                author = input('Authors name:')
                title = s.search_author(author)

            if search_input == 'title':
                title = input('Title name:')
                title = s.search_title(title)

            if search_input == 'both':
                author = input('Authors name:')
                title = input('Title name:')
                filters = {'Author': author,"Extension": "pdf"}
                title = s.search_title_filtered(title, filters, exact_match=True)

            if (search_input == 'exit'):
                break
                
            if search_input not in ["author", "title", "both", "exit"]:
                search_input = input('Please type a valid response (author, title, or both) ')
            
            if len(title) == 0:
                print('No books found')
                continue
            if len(title) >= 1:
                x = 0
                for item in title:
                    print('index_num: ', x, '\n', 
                          'Author: ' + item['Author'],  '\n', 
                          'Title: ' +  item['Title'], '\n',
                          'Year: ' +  item['Year'], '\n', 
                          'Pages: ' +  item['Pages'], '\n', 
                          'Language: ' + item['Language'], '\n',
                          'Extension: ' + item['Extension'], '\n',
                          'Size: ' + item['Size'], '\n')
                    x+=1
                selection = int(input('Enter index number of version you want to keep (-1 for none)'))
                if selection == -1:
                    continue
                df = df.append(title[selection], ignore_index = True)
                end_case = input('Do you want to find another book? (y/n) ')
                if end_case == 'y':
                    #print('\n')
                    clear_output()
                    continue
                if end_case == 'n':
                    break
                while(end_case != 'n') and (end_case != 'y'):
                    end_case = input('Please type a valid response (y/n) ')
                    clear_output()
        df['Title_text'] = ''
        return df
        
    except Exception: 
        print('Error: ', traceback.format_exc())
        return df

In [138]:
def scrape_book_text(pdf_list, df, index_list):
    x = 0
    while x < len(pdf_list):
        def create_book_df(filepath: str) -> pd.DataFrame():
            """Extracts text from a pdf and puts it into a dataframe
                (assuming its a book)

            Args:
                filepath (str): Path to file

            Returns:
                [pd.Dataframe()]: Pandas Dataframe of the book:
                                    (page_number, page_text)
            """
            try:
                with fitz.open(filepath) as doc:
                    page_texts = [
                        (page_num + 1, page.get_text()) 
                        for page_num, page in enumerate(doc) if page.get_text() not in ["", None]
                    ]
                    return pd.DataFrame(page_texts, columns = ['Page Number', "Page Text"])
            except Exception as e:
                pass
        book = create_book_df('book_pdf/' + pdf_list[x] + '.pdf')
        book = ''.join(book['Page Text'])
        df['Title_text'][index_list[x]] = book 
        x+=1
    return df

In [139]:
# This runs the libgen scraper t
df = scrape_books()

How do you want to search? (author, title, or both) both
Authors name: Edward W. Said
Title name: Orientalism


index_num:  0 
 Author: Edward W. Said 
 Title: Orientalism 
 Year: 1979 
 Pages: 234 
 Language: English 
 Extension: pdf 
 Size: 2 Mb 

index_num:  1 
 Author: Edward W. Said 
 Title: Orientalism: Western Conceptions of the Orient 
 Year: 1979 
 Pages: 368[378] 
 Language: English 
 Extension: pdf 
 Size: 10 Mb 

index_num:  2 
 Author: Edward W. Said 
 Title: Orientalism: Western Conceptions of the Orient (Penguin Modern Classics) 
 Year: 2003 
 Pages: 432[414] 
 Language: English 
 Extension: pdf 
 Size: 6 Mb 

index_num:  3 
 Author: Edward W. Said 
 Title: Orientalism 
 Year: 1979 
 Pages: 368[378] 
 Language: English 
 Extension: pdf 
 Size: 10 Mb 

index_num:  4 
 Author: Edward W. Said 
 Title: Orientalism 
 Year: 1979 
 Pages: 368[378] 
 Language: English 
 Extension: pdf 
 Size: 10 Mb 



Enter index number of version you want to keep (-1 for none) 0
Do you want to find another book? (y/n)  n


In [140]:
# Print this to show the dataframe collected after running `scrape_books()`
df 

,ID,Author,Title,Publisher,Year,Pages,Language,Size,Extension,Mirror_1,Mirror_2,Mirror_3,Mirror_4,Mirror_5,Edit,Title_text
0,1339914,Creek Stewart,The Unofficial Hunger Games Wilderness Surviva...,F+W Media,2013,,English,26 Mb,epub,http://library.lol/main/59E9C9E8D140ADA9319DBE...,https://cdn1.booksdl.org/ads.php?md5=59E9C9E8D...,https://3lib.net/md5/59E9C9E8D140ADA9319DBE9C4...,https://library.bz/main/edit/59E9C9E8D140ADA93...,NaN,NaN,
1,489188,Nicholas Dew,Orientalism in Louis XIV's France (Oxford Hist...,,2009,320[318],English,2 Mb,pdf,http://library.lol/main/C348C04399A96A9C071293...,https://cdn1.booksdl.org/ads.php?md5=C348C0439...,https://3lib.net/md5/C348C04399A96A9C071293FE7...,https://library.bz/main/edit/C348C04399A96A9C0...,NaN,NaN,
2,269195,Edward W. Said,Orientalism,Penguin Books,1979,234,English,2 Mb,pdf,http://library.lol/main/628C1111E4BC21EF54A74B...,https://cdn1.booksdl.org/ads.php?md5=628C1111E...,https://3lib.net/md5/628C1111E4BC21EF54A74B76C...,https://library.bz/main/edit/628C1111E4BC21EF5...,NaN,NaN,


In [121]:
df_gender_studies = df

In [123]:
df_gender_studies.to_csv('book_dataframes/gender_studies_books.csv')

In [ ]:
df.to_csv('post_colonial_books.csv')

In [ ]:
# Use these lists if you are adding more than one book
pdf_list = ['Culture and imperialism by Edward W. Said (z-lib.org)',
           'Black Skin, White Masks by Frantz Fanon (z-lib.org)',
           'The location of culture by Homi K. Bhabha (z-lib.org)',
           'Black Athena The Afroasiatic Roots of Classical Civilization The Linguistic Evidence, Vol. 3 by Martin Bernal (z-lib.org)',
           'The World, the Text, and the Critic by Edward W. Said (z-lib.org)',
           'Bruce Robbins - Perpetual War_ Cosmopolitanism from the Viewpoint of Violence-Duke University Press (2012)']
index_list = [0,3,4,5,6,7]

In [ ]:
df = scrape_book_text(pdf_list, df, index_list)

In [ ]:
df

In [ ]:
df.to_csv('book_dataframes/post_colonial_books.csv')